In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import soundfile as sf


def feature_extraction(file_name):

    X , sample_rate = librosa.load(file_name, sr=None)
    if X.ndim > 1:
        X = X[:,0]
    X = X.T
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=20).T, axis=0)
    rmse = np.mean(librosa.feature.rms(y=X).T, axis=0)
    spectral_flux = np.mean(librosa.onset.onset_strength(y=X, sr=sample_rate).T, axis=0)
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=X).T, axis=0)
    return mfccs, rmse, spectral_flux, zcr

def parse_audio_files(parent_dir, sub_dirs, file_ext='*.wav'):
    n_mfccs = 20
    number_of_features = 3 + n_mfccs
    features, labels = np.empty((0,number_of_features)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for file_name in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            print ("Actual File Name: ", file_name)
            try:
                mfccs, rmse, spectral_flux, zcr = feature_extraction(file_name)
            except Exception as e:
                print("[Error] there was an error in feature extraction. %s" % (e))
                continue
            extracted_features = np.hstack([mfccs, rmse, spectral_flux, zcr])
            features = np.vstack([features, extracted_features])
            labels = np.append(labels, label)
        print("Extracted features from %s, done" % (sub_dir))
    return np.array(features), np.array(labels, dtype =int)


In [ ]:
cd /content/drive/MyDrive

In [ ]:
#Read audio classes directories
import os
audio_subdirectories = os.listdir("audio data")
audio_subdirectories.sort()
print ('Audio Subdirs: ', audio_subdirectories)

In [ ]:
from os import walk
from os import path

for (dirpath, dirnames, filenames) in walk("/content/drive/MyDrive/audio data/001 - Low/"):
    print(filenames)
    for i in range(len(filenames)):
        old_path = os.path.join("/content/drive/MyDrive/audio data/001 - Low/", filenames[i])
        new_path = os.path.join("/content/drive/MyDrive/audio data/001 - Low/", filenames[i].replace(" ", "_") + '.wav')
        os.rename(old_path, new_path)

In [ ]:
import os
from os import walk
from os import path

for (dirpath, dirnames, filenames) in walk("/content/drive/MyDrive/audio data/003 - High"):
    print(filenames)
    for i in range(len(filenames)):
        old_path = os.path.join("/content/drive/MyDrive/audio data/003 - High", filenames[i])
        new_path = os.path.join("/content/drive/MyDrive/audio data/003 - High", filenames[i].replace(" ", "_") + '.wav')
        os.rename(old_path, new_path)

In [ ]:
features, labels = parse_audio_files('/content/drive/MyDrive/audio data',audio_subdirectories)
np.save('feat6.npy', features)
np.save('label6.npy', labels)

print (len(features))

#Load np files
import numpy as np
features= np.load('feat4.npy')
print(len(features[1]))

import numpy as np
features= np.load('feat4.npy')

#Pandas dataframe with N features for each audio
import pandas as pd
df = pd.DataFrame(features)
df

labels = np.load('label5.npy')
print(labels)

# For future label de-encoding
label_classes = np.array(['Low','Intermediate','High'])
print(label_classes)

np.unique(labels)

In [ ]:
import matplotlib.pyplot as plt
import librosa.display
plt.figure(figsize=(13, 6))
librosa.display.specshow(features, x_axis='time')
plt.colorbar()
plt.title('MFCCs = 5 Values for 5s Audio Frames (High Class)')
plt.tight_layout()
plt.savefig('test saving.png')

In [ ]:
#visulasisation of spectrogram
import matplotlib.pyplot as plt
y , sr = librosa.load('/content/drive/MyDrive/audio data/001 - Low/Avalinguo_-_Dana_and_Konay_segment_102_-_D.wav.wav.wav', sr=None) # Using my own audio segment
plt.figure(figsize=(20, 15))
D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
librosa.display.specshow(D, y_axis='linear')
#plt.colorbar(format='%+2.0f dB')
#plt.title('Linear-frequency power spectrogram')
#plt.subplot(4, 2, 2)
librosa.display.specshow(D, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')
C = librosa.feature.chroma_cqt(y=y, sr=sr)
plt.subplot(4, 2, 3)
librosa.display.specshow(C, y_axis='chroma')
plt.colorbar()
plt.title('Chromagram')
plt.subplot(4, 2, 4)
librosa.display.specshow(D, cmap='gray_r', y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Linear power spectrogram (grayscale)')

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

# Fix random seed number # for the reproducapility purpose
np.random.seed(7)

# Load the data
X = np.load('feat5.npy')
y = np.load('label5.npy').ravel()

number_of_features = len(X[1])
number_of_classes = 3

# Sample data randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Neural Network Architecture
model = Sequential()

# 1st Layer
model.add(Dense(512, input_dim=number_of_features, activation='relu'))
model.add(Dropout(0.5))

# 2nd Layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# 3rd Layer. Output 3 neurons corresponding to the number of classes
model.add(Dense(number_of_classes, activation='softmax'))

# Model Compilation. Loss for multi-class classification problem
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)  # Use learning_rate instead of lr
rmsprop = 'rmsprop'
adam = 'adam'
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,  # rmsprop better than sgd
              metrics=['accuracy'])

# Convert labels to categorical one-hot encoding
y_train = keras.utils.to_categorical(y_train-1, num_classes=number_of_classes)
y_test = keras.utils.to_categorical(y_test-1, num_classes=number_of_classes)

# Train and test
model.fit(X_train, y_train, epochs=300, batch_size=64)  # batch 32, 64, 128, 256, 512
score, acc = model.evaluate(X_test, y_test, batch_size=64)


In [ ]:
print ('Test score:', score)
print('Test accuracy:', acc)

Saving the Model

In [ ]:
import pickle

In [ ]:
from tensorflow.keras.models import save_model


# Save the model in HDF5 format
model.save('classifier.h5')

In [ ]:
!pwd

In [ ]:
from google.colab import files

files.download('classifier.h5')